Delayed Labeling Streaming Classification | Airline Datensatz

In [6]:
import numpy as np
import pandas as pd

from skmultiflow.trees import HoeffdingTreeClassifier
from skmultiflow.trees import HoeffdingAdaptiveTreeClassifier
from skmultiflow.trees import ExtremelyFastDecisionTreeClassifier
from skmultiflow.meta import AdaptiveRandomForestClassifier
from skmultiflow.bayes import NaiveBayes
from skmultiflow.meta import AccuracyWeightedEnsembleClassifier
from skmultiflow.lazy import KNNADWINClassifier

from skmultiflow.data import TemporalDataStream

from skmultiflow.evaluation import EvaluatePrequentialDelayed


In [7]:
import warnings
warnings.filterwarnings('ignore')

# set column
attribute = ["Airline","Flight","AirportFrom","AirportTo","DayOfWeek","Time","Length"]
target = "Delay"
timestamp = "datetime"
nominal=['Airline', 'Flight', 'AirportFrom', 'AirportTo', 'DayOfWeek']

# Read a csv with stream data
air_t = pd.read_csv("data/air_t.csv")
# Convert time column to datetime
air_t[timestamp] = pd.to_datetime(air_t[timestamp])

# Sort data by time
air_t = air_t.sort_values(by=timestamp)

# Get X, y and time
X = air_t[attribute].values
y = air_t[target].values
time = air_t[timestamp].values

# set a delay of 30 min
delay_time = np.timedelta64(1, "D")

# create strem
air_t_stream = TemporalDataStream(X, y, time, sample_delay=delay_time, ordered=False)

ht = HoeffdingTreeClassifier(leaf_prediction='nb', nominal_attributes=nominal)
hat = HoeffdingAdaptiveTreeClassifier(nominal_attributes=nominal)
efdt = ExtremelyFastDecisionTreeClassifier(nominal_attributes=nominal)
arf = AdaptiveRandomForestClassifier(nominal_attributes=nominal)
snb = NaiveBayes(nominal_attributes=nominal)
awe = AccuracyWeightedEnsembleClassifier(n_estimators=15, base_estimator=NaiveBayes(nominal_attributes=nominal))
knn_adwin = KNNADWINClassifier(n_neighbors=10)

evaluator = EvaluatePrequentialDelayed(batch_size=1,
                                n_wait=10,
                                pretrain_size=100,
                                max_samples=X.shape[0],
                                output_file='results_air_t.csv',
                                metrics=['accuracy', 'kappa', 
                                         'precision', 'recall', 
                                         'f1', 'running_time', 
                                         'model_size'])

evaluator.evaluate(stream=air_t_stream, 
                   model=[ht, hat, efdt, arf, 
                          snb, awe, knn_adwin], 
                   model_names=['Hoeffding Tree', 'Hoeffding Adaptive Tree', 
                                'Extremely Fast Decision Tree', 
                                'Adaptive Random Forest', 
                                'Naive Bayes', 'Accuracy Weighted Ensembler', 'KNN ADWIN' ])

Prequential Evaluation Delayed
Evaluating 1 target(s).
Pre-training on 100 sample(s).
Evaluating...
 ###################- [95%] [518.83s]Processed samples: 10000
Mean performance:
Hoeffding Tree - Accuracy     : 0.5941
Hoeffding Tree - Kappa        : 0.1657
Hoeffding Tree - Precision: 0.5117
Hoeffding Tree - Recall: 0.5173
Hoeffding Tree - F1 score: 0.5145
Hoeffding Tree - Training time (s)  : 1.62
Hoeffding Tree - Testing time  (s)  : 1.89
Hoeffding Tree - Total time    (s)  : 3.51
Hoeffding Tree - Size (kB)          : 52.4365
Hoeffding Adaptive Tree - Accuracy     : 0.5843
Hoeffding Adaptive Tree - Kappa        : 0.0000
Hoeffding Adaptive Tree - Precision: 0.0000
Hoeffding Adaptive Tree - Recall: 0.0000
Hoeffding Adaptive Tree - F1 score: 0.0000
Hoeffding Adaptive Tree - Training time (s)  : 6.66
Hoeffding Adaptive Tree - Testing time  (s)  : 0.61
Hoeffding Adaptive Tree - Total time    (s)  : 7.28
Hoeffding Adaptive Tree - Size (kB)          : 79.6064
Extremely Fast Decision Tree - 

[HoeffdingTreeClassifier(binary_split=False, grace_period=200,
                         leaf_prediction='nb', max_byte_size=33554432,
                         memory_estimate_period=1000000, nb_threshold=0,
                         no_preprune=False,
                         nominal_attributes=['Airline', 'Flight', 'AirportFrom',
                                             'AirportTo', 'DayOfWeek'],
                         remove_poor_atts=False, split_confidence=1e-07,
                         split_criterion='info_gain', stop_mem_management=False,
                         tie_threshold=0.05),
 HoeffdingAdaptiveTreeClassifier(binary_split=False, bootstrap_sampling=True,
                                 grace_period=200, leaf_prediction='nba',
                                 max_byte_size=33554432,
                                 memory_estimate_period=1000000, nb_threshold=0,
                                 no_preprune=False,
                                 nominal_attributes=['